## Laboratorio 4

Joaquin Puente
José Mérida

In [ ]:
# ===== Bitácoras en silencio =====
import os
# Reduce el ruido de bitácoras del TF (0 = todo, 1 = INFO apagado, 2 = +WARNING apagado, 3 = +ERROR apagado)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [ ]:
# ===== 1) Importaciones & chequeo del ambiente =====
import tensorflow as tf
from tensorflow.keras import mixed_precision
import numpy as np, platform

# Mantener el entrenamiento estably y predecible en los procesadores M (Metal)
mixed_precision.set_global_policy("float32")

print("Python:", platform.python_version(), "arch:", platform.machine())
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

# Redes Neuronales Convolucionales (CNN) para la clasificación de imágenes

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

##  Visualización de los datos de las imágenes

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X_train.shape

In [ ]:
una_imagen = X_train[0]

In [ ]:
una_imagen

In [ ]:
una_imagen.shape

In [ ]:
plt.imshow(una_imagen)

¿Qué está pasando acá?   ¿No se supone que es una imagen B-N?

Lo que pasa es que Matplotlib tiene una variedad de esquemas de colores "colormaps".  Si se desea, se puede cambiar el esquema para que se vea en B-N

# Pre-procesamiento de los datos

Es necesario asegurar que las etiquetas (metas) sean comprensibles por la CNN

## Etiquetas

In [ ]:
y_train

In [ ]:
y_test

Parece que las etiquetas son literalmente **categorías numéricas**, pero están en formato numérico.  Será necesario convertirlas por el método de "one hot encoding" para que puedan ser usadas por la CNN, de lo contrario pensará que es algún tipo de problema de regresión sobre un eje contínuo.

Afortunadamente, Keras tiene una función fácil para hacer esta conversión:

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train.shape

In [ ]:
ejemplo_y = to_categorical(y_train)

In [ ]:
ejemplo_y

In [ ]:
ejemplo_y.shape

In [ ]:
ejemplo_y[0]

el método to_categorical puede inferir, por default, el número de clases...y lo hace bastante bien.  Sin embargo, si se quiere estar seguro (podría ser que los datos no tuvieran uno de los valores posibles), o si fuera un caso más complicado, se puede especificar.  En este caso son 10

In [ ]:
y_cat_test = to_categorical(y_test, 10)

In [ ]:
y_cat_train = to_categorical(y_train, 10)

### Pre-Procesamiento de los datos X

Es mejor normalizar los datos de X

Normalmente se haría con el método MinMax() de sklearn, porque no se puede asumir que se sabe qué valores mínimo y máximo podrían venir en los datos futuros que quieran clasificar.  Sin embargo, como este ejercicio trata de imágenes, sí se conoce que todos los valores serán entre 0 y 255 por lo que se puede tomar una salida fácil.

In [ ]:
una_imagen.max()

In [ ]:
una_imagen.min()

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
una_normalizada = X_train[0]

In [ ]:
una_normalizada.max()

In [ ]:
plt.imshow(una_normalizada)

Se puede ver que es exactamente igual!

## Cambiar el formato de los datos

Ahorita los datos son 60,000 imágenes almacenadas en un formato de 28 X 28 pixeles.

Esto es correcto para una CNN, pero es necesario agregar una dimensión más para mostrar que se está trabajando con un canal RGB (ya que, técnicamente, las imágenes están en B-N, y solo muestran valores entre 0-255 en un solo canal).  Una imagen a colores tendría 3 canales o dimensiones.

In [ ]:
X_train.shape

In [ ]:
X_test.shape

Modificar la forma para incluir la dimensión correspondiente al canal (en este caso es 1, cuando sea a colores es 3)

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1)

In [ ]:
X_train.shape

In [ ]:
X_test = X_test.reshape(10000, 28, 28, 1)

In [ ]:
X_test.shape

# Diseño del modelo

Se hacen importaciones para poder crear capas normales, capas convolucionales, capas de pooling, y se necesita una capa para "aplanar" los datos (a una sola dimensión)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [ ]:
modelo = Sequential()

## Capa de entrada

En versiones previas de Tensorflow, el formato de los datos de entrada se incluían en la primera capa definida.  Ahora se recomienda como buena práctica incluir una capa específica de entrada de datos.

In [ ]:
modelo.add(Input(shape=(28, 28, 1)))

### Capa Convolucional

El número de filtros es configurable, generalmente se usa un múltiplo de 2.  El tamaño también es configurable, sin embargo es bueno ajustarlo al tamaño de la imágen.  Cómo el filtro se irá "corriendo", es bueno que el tamaño de la imagen sea un múltiplo del tamaño del filtro

In [ ]:
# Sintáxis anterior para primera capa
#modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), input_shape = (28, 28, 1), activation = 'relu'))

# Recomendación de mejores prácticas (sin formato de entrada)
modelo.add(Conv2D(filters = 32, kernel_size = (4, 4), activation = 'relu'))

### Capa de sub-muestreo (Pooling)

In [ ]:
modelo.add(MaxPool2D(pool_size = (2, 2)))

### Capa parar aplanar 

Antes de llegar a la capa final es importante "aplanar" de 28 X 28 a 764 

In [ ]:
modelo.add(Flatten())

### Capas escondidas

Tendrá 128 "neuronas" o unidades (este valor es configurable)

In [ ]:
modelo.add(Dense(128, activation = 'relu'))

### La última capa es la clasificadora

Tenemos 10 posibles clases por lo que se usa la función de activación "softmax" que es para multi-clases

In [ ]:
modelo.add(Dense(10, activation = 'softmax'))

### Se compila el modelo

Como una opción se pueden pedir una o más métricas, para ver cuáles hay, se puede consultar en:

https://keras.io/metrics/

In [ ]:
modelo.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
modelo.summary()

### Uso de "callbacks"

Una forma de detener las épocas es usando el EarlyStopping.  Esto tiene un parámetro que se denomina "patience" que, como dice su nombre indica el grado de paciencia que debe tener, el modelo, una vez se detecta que el parámetro monitoreado empiece a desviarse de lo deseado.  La paciencia se mide en épocas, muchos usan 2 o 3 para dar chance a que el ultimo valor no haya sido alguna anomalía, y darle otra oportunidad.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
detencion_temprana = EarlyStopping(monitor = 'val_loss', patience = 2)

## Entrenar el modelo

In [ ]:
tamanio_tanda = 32

modelo.fit(X_train, y_cat_train, 
           batch_size = tamanio_tanda,
           epochs = 10, 
           validation_data = (X_test,y_cat_test),
           callbacks = [detencion_temprana],
           verbose = 2)

## Evaluar el modelo

Para saber qué métricas hay disponibles:

In [ ]:
modelo.metrics_names

In [ ]:
metricas = pd.DataFrame(modelo.history.history)

In [ ]:
metricas.head()

In [ ]:
metricas[['accuracy', 'val_accuracy']].plot()

In [ ]:
metricas[['loss', 'val_loss']].plot()

In [ ]:
print(modelo.metrics_names)
print(modelo.evaluate(X_test, y_cat_test, verbose = 0))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Hasta hace poco, en Tensorflow se podía utilizar la instrucción:
    
    predicciones = modelo.predict_classes(X_test)
    
En las versiones más recientes de Tensorflow debe usarse:
    
Classification multiclase:
    
predicciones = np.argmax(modelo.predict(X_test), axis=-1)

Classification Binaria

predicciones = (modelo.predict(X_test) > 0.5).astype("int32")

In [ ]:
#predicciones = modelo.predict_classes(X_test)
predicciones = np.argmax(modelo.predict(X_test), axis=-1)

In [ ]:
y_cat_test.shape

In [ ]:
y_cat_test[0]

In [ ]:
predicciones[0]

In [ ]:
y_test

In [ ]:
print(classification_report(y_test, predicciones))

In [ ]:
confusion_matrix(y_test, predicciones)

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize = (10, 6))
sns.heatmap(confusion_matrix(y_test,predicciones),annot = True)

# Predecir una imagen dada

In [ ]:
mi_numero = X_test[6]

In [ ]:
plt.imshow(mi_numero.reshape(28,28))

Recordar que la forma debe ser:  (num_imagenes, ancho, alto, num_canales_color)

In [ ]:
np.argmax(modelo.predict(mi_numero.reshape(1,28,28,1)), axis=-1)
#modelo.predict_classes(mi_numero.reshape(1,28,28,1))

Parece que la CNN funciona bastante bien!